In [1]:
from kiteconnect import KiteConnect
import time
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

In [2]:
kite = KiteConnect(api_key="3np0d05xqq02k905")
kite.set_access_token("D2KQUc0F4Wl7KEUfcddRpslWTKgOIMNk")

In [3]:

df_instruments=pd.DataFrame(kite.instruments())

In [23]:
df_historical=pd.DataFrame(kite.historical_data(738561,"2022-12-01 00:00:00","2023-02-02 00:30:00","day"))

In [24]:
fig = go.Figure(data=[go.Candlestick(x=df_historical['date'],
                open=df_historical['open'],
                high=df_historical['high'],
                low=df_historical['low'],
                close=df_historical['close'])])
fig.show()

In [14]:
df_historical

""


In [ ]:

# Place an order
order_id = kite.place_order(tradingsymbol="INFY",
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            variety=kite.VARIETY_AMO,
                            order_type=kite.ORDER_TYPE_MARKET,
                            product=kite.PRODUCT_CNC,
                            validity=kite.VALIDITY_DAY)



# Fetch all orders
kite.orders()

# Get instruments
kite.instruments()

# Place an mutual fund order
kite.place_mf_order(
    tradingsymbol="INF090I01239",
    transaction_type=kite.TRANSACTION_TYPE_BUY,
    amount=5000,
    tag="mytag"
)

# Cancel a mutual fund order
kite.cancel_mf_order(order_id="order_id")

# Get mutual fund instruments
kite.mf_instruments()